In [8]:
!nvidia-smi

Thu Jun 30 14:40:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8    13W /  N/A |   1607MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
torch.version.cuda

'11.3'

In [7]:
x_ones = torch.ones(3)
x_ones.to("cuda")

tensor([1., 1., 1.], device='cuda:0')

In [3]:
from T5FineTuner import RPDataset
from utils import get_folds
import torch
import argparse
from transformers import T5Tokenizer, AutoTokenizer, AutoModelForSeq2SeqLM
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb

DATASETS = ["RP-Crowd-3", "RP-Crowd-2", "RP-Mod"]
# suffix = "german-t5-oscar-ep1-prompted-germanquad"
model_names = ["t5-efficient-gc4-german-small-el32", "t5-base-german-3e"]
# MODEL_NAME_OR_PATH = f"GermanT5/{suffix}"
# WANDB_PROJECT_NAME = f"all-datasets-{suffix}"
# OUTPUT_DIR = f"./{suffix}"

# auto_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
# auto_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)


[nltk_data] Downloading package punkt to /home/dobby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
[nltk_data] Downloading package punkt to /home/dobby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import csv

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

# eval packages
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

from torch.optim import AdamW

from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)



class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    # self.hparams.save_hyperparameters(hparams)
    self.save_hyperparameters(hparams)
    
    self.model = hparams.model
    self.tokenizer = hparams.tokenizer
    # self.train_dataset = train_dataset
    # self.val_dataset = val_dataset
    
  def is_logger(self):
    return self.trainer.global_rank <= 0
  

  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
        # lm_labels=lm_labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        # lm_labels=lm_labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]

    return loss
  
  def get_accuracy(self, batch):
    labels = batch["target_ids"]

    outs = self.model.generate(input_ids=batch["source_ids"], 
                                attention_mask=batch["source_mask"], 
                                max_length=2)

    dec = [self.tokenizer.decode(ids) for ids in outs]
    target = [self.tokenizer.decode(label) for label in labels]

    new_outputs = [s[6:] for s in dec]
    new_targets = [s[:-4] for s in target]

    accuracy_score = metrics.accuracy_score(new_targets, new_outputs)
    f1 = metrics.f1_score(new_targets, new_outputs, labels=["problematisch"], average=None)
    recall = metrics.recall_score(new_targets, new_outputs, labels=["problematisch"], average=None)
    precision = metrics.precision_score(new_targets, new_outputs, labels=["problematisch"], average=None)
    # rec = metrics.recall_score(new_targets, new_outputs, average="micros")
    return accuracy_score, f1, recall, precision
  # def computer_accuracy(self):
  #   self.model.model.eval()

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log("train/loss", loss)
    return {"loss": loss}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    self.log("avg_train_loss", avg_train_loss)
    
  def validation_step(self, batch, batch_idx):
    
    loss = self._step(batch)
    accuracy, f1, recall, precision = [torch.tensor(val) for val in self.get_accuracy(batch)]
    self.log("val/loss", loss, logger=True)
    self.log("val/accuracy", accuracy, logger=True)
    self.log("val/f1", f1, logger=True)
    self.log("val/recall", recall, logger=True)
    self.log("val/precision", precision, logger=True)
    return {"val/loss": loss, "val/accuracy": accuracy, "val/f1": f1, "val/recall": recall, "val/precision": precision}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val/loss"] for x in outputs]).mean()
    avg_accuracy = torch.stack([x["val/accuracy"] for x in outputs]).mean()
    avg_f1 = torch.stack([x["val/f1"] for x in outputs]).mean()
    avg_recall = torch.stack([x["val/recall"] for x in outputs]).mean()
    avg_precision = torch.stack([x["val/precision"] for x in outputs]).mean()
    # tensorboard_logs = {"val_loss": avg_loss}
    self.log("avg_val_loss", avg_loss)
    self.log("avg_val_accuracy", avg_accuracy)
    self.log("avg_val_f1", avg_f1)
    self.log("avg_val_recall", avg_recall)
    self.log("avg_val_precision", avg_precision)
    # self.log("log", tensorboard_logs)
    # self.log("progress_bar", tensorboard_logs)
    # self.log({"avg_val_loss": avg_loss, 
    #           "log": tensorboard_logs,
    #           'progress_bar': tensorboard_logs}, logger=True, prog_bar=True)
    return {"avg_val_loss": avg_loss, "avg_val_accuracy": avg_accuracy}

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, \
                      eps=self.hparams.adam_epsilon, betas=self.hparams.adam_betas)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self, 
                      epoch,
                      batch_idx,
                      optimizer,
                      optimizer_idx,
                      second_order_closure=None,
                      on_tpu=None,
                      using_native_amp=None,
                      using_lbfgs=None):
    # if self.trainer.use_tpu:
    #   xm.optimizer_step(optimizer)
    # else:
    optimizer.step(closure=second_order_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()
  
  # def closure(self):
  #   return "closure"
  
  def get_tqdm_dict(self):
    tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

    return tqdm_dict

  def train_dataloader(self):
    # train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(self.hparams.train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    # val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(self.hparams.val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

[nltk_data] Downloading package punkt to /home/dobby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Setting up Automatic Hyperparameter tuning

### Normal Training Loop

In [5]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [6]:
DATASETS = ["RP-Crowd-3", "RP-Crowd-2", "RP-Mod"]
# suffix = "german-t5-oscar-ep1-prompted-germanquad"
model_names = ["t5-efficient-oscar-german-small-el32"]
learning_rates = [1e-4, 4e-4, 5.6e-5]

for suffix in model_names:
    MODEL_NAME_OR_PATH = "GermanT5/t5-base-german-3e"
    WANDB_PROJECT_NAME = f"all-datasets-{suffix}"
    OUTPUT_DIR = f"./{suffix}"

    auto_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
    
    for dataset in DATASETS:
        source = f"./Datasets/{dataset}-folds.csv"
        train_inputs, train_targets, val_inputs, val_targets = get_folds(source)

        train_dataset = RPDataset(auto_tokenizer, train_inputs, train_targets)
        valid_dataset = RPDataset(auto_tokenizer, val_inputs, val_targets)
        for lr in learning_rates:
            run_name = f"{dataset}-lr-{lr}"

            # wandb.finish()

            
            # wandb_logger = WandbLogger(project=WANDB_PROJECT_NAME, 
            # name=run_name)

            # wandb.define_metric("val/accuracy", summary="max")
            # wandb.define_metric("val/f1", summary="max")

            checkpoint_callback = pl.callbacks.ModelCheckpoint(
                dirpath=OUTPUT_DIR + run_name, filename="{epoch}-{val/accuracy:.2f}-{val/loss:.2f}", monitor="val/accuracy", mode="max", save_top_k=5
            )

            auto_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH, from_flax=True)
            args_dict = dict(
                    data_dir="", # path for data files
                    output_dir=f"./GermanT5-RP-Mod/t5-efficient-oscar-german-small-el32/", # path to save the checkpoints
                    model_name_or_path=MODEL_NAME_OR_PATH,
                    tokenizer_name_or_path=MODEL_NAME_OR_PATH,
                    dataset_name=dataset,
                    max_seq_length=512,
                    learning_rate=lr,
                    weight_decay=0.1,
                    adam_epsilon=1e-8,
                    adam_betas=(0.9,0.999),
                    warmup_steps=0,
                    train_batch_size=4,
                    eval_batch_size=2,
                    num_train_epochs=10,
                    gradient_accumulation_steps=1,
                    n_gpu=1,
                    early_stop_callback=False,
                    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
                    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
                    max_grad_norm=0.5, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
                    seed=42,
                    train_dataset=train_dataset, 
                    val_dataset=valid_dataset, 
                    model=auto_model,
                    tokenizer=auto_tokenizer
                )
            
            args = argparse.Namespace(**args_dict)

            train_params = dict(
                        accumulate_grad_batches=args.gradient_accumulation_steps,
                        auto_lr_find=True,
                        gpus=args.n_gpu,
                        max_epochs=args.num_train_epochs,
                        default_root_dir=f"/home/dobby/{MODEL_NAME_OR_PATH}",
                        # early_stop_callback=False,
                        precision= 16 if args.fp_16 else 32,
                        amp_level=args.opt_level,
                        gradient_clip_val=args.max_grad_norm,
                        # checkpoint_callback=checkpoint_callback,
                        logger=wandb_logger,
                        enable_checkpointing=checkpoint_callback,
                        callbacks=[checkpoint_callback],
                        # callbacks=[raytuner_callback],
                        # callbacks=[LoggingCallback()],
                        amp_backend="apex"
                    )
            
            model = T5FineTuner(args)
            trainer = pl.Trainer(**train_params)

            trainer.fit(model)

Loading a Flax model in PyTorch, requires both PyTorch and Flax to be installed. Please see https://pytorch.org/ and https://flax.readthedocs.io/en/latest/installation.html for installation instructions.


ModuleNotFoundError: jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.

In [ ]:
%pip install jax

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 9.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 36.4 MB/s eta 0:00:00
  Created wheel for jax: filename=jax-0.3.14-py3-none-any.whl size=1147584 sha256=263e8aae2fa1cea6edb19a7e9163c4337d2c6ad9cdf4df7dae932271e1bcb97a
  Stored in directory: /home/dobby/.cache/pip/wheels/32/21/2b/29f2d0dba28673825c67ce8451e44b07ca7bbf8e68964a82db
Successfully built jax
Note: you may need to restart the kernel to use updated packages.


In [7]:
print(torch.version.cuda)

10.2
